In [ ]:
# ❌ VULNERABILIDAD 1: API Keys hardcoded
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle
import sqlite3

# API Keys expuestas
OPENAI_API_KEY = os.getenv("GENERIC_API_KEY")
AWS_ACCESS_KEY = "[REDACTED_SECRET]"
[REDACTED_SECRET]
STRIPE_SECRET_KEY = "[REDACTED_SECRET]"

print(f"✅ API Keys cargados: {OPENAI_API_KEY[:10]}...")

In [ ]:
# ❌ VULNERABILIDAD 2: Database credentials hardcoded
DB_HOST = "mi-banco-prod.us-east-1.rds.amazonaws.com"
DB_USER = "admin"
DB_PASSWORD = os.getenv("PASSWORD")
DB_NAME = "banking_prod"

# Connection string expuesto
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:5432/{DB_NAME}"
print(f"Conectando a: {connection_string}")

In [ ]:
# ❌ VULNERABILIDAD 3: SQL Injection
def get_customer_data(customer_id):
    """Obtiene datos de cliente - VULNERABLE A SQL INJECTION"""
    conn = sqlite3.connect('banking.db')
    
    # SQL query vulnerable - usa string concatenation
    query = f"SELECT * FROM customers WHERE customer_id = '{customer_id}'"
    
    df = pd.read_sql(query, conn)
    conn.close()
    return df

# Un atacante podría pasar: "1' OR '1'='1"
customer_id = input("Ingrese customer ID: ") if False else "12345"
data = get_customer_data(customer_id)
print(data.head())

In [ ]:
# ❌ VULNERABILIDAD 4: PII expuesta en logs y visualizaciones
# Crear datos de ejemplo con PII real
customers = pd.DataFrame({
    'customer_id': range(1, 101),
    'name': [f'Cliente {i}' for i in range(1, 101)],
    'email': [f'cliente{i}@email.com' for i in range(1, 101)],
    'dni': [f'{10000000 + i}' for i in range(1, 101)],
    'phone': [f'+51 999 {i:03d} {i*2:03d}' for i in range(1, 101)],
    'account_number': [f'ACC-{i:010d}' for i in range(1, 101)],
    'income': np.random.randint(1000, 10000, 100),
    'age': np.random.randint(18, 70, 100),
    'gender': np.random.choice(['M', 'F'], 100),
    'credit_score': np.random.randint(300, 850, 100)
})

# ❌ Imprimir PII en logs
print("\n=== DATOS DE CLIENTES (PII EXPUESTA) ===")
print(customers.head(20))  # Muestra DNI, email, teléfono, etc.

# ❌ PII en visualización
print(f"\nCliente ejemplo: {customers.iloc[0]['name']}")
print(f"DNI: {customers.iloc[0]['dni']}")
print(f"Email: {customers.iloc[0]['email']}")
print(f"Teléfono: {customers.iloc[0]['phone']}")

In [ ]:
# ❌ VULNERABILIDAD 5: Data Leakage
# Crear target (aprobación de crédito)
customers['credit_approved'] = (customers['credit_score'] > 600).astype(int)

# ❌ DATA LEAKAGE: Usar información del futuro
customers['future_default'] = np.random.choice([0, 1], 100, p=[0.9, 0.1])  # Esto NO existiría al momento de decisión
customers['total_payments_made'] = np.random.randint(0, 24, 100)  # Pagos futuros

# ❌ DATA LEAKAGE: Target leakage - usar variable derivada del target
customers['risk_category'] = customers['credit_approved'].map({0: 'HIGH_RISK', 1: 'LOW_RISK'})

print("Features con data leakage:")
print(customers[['credit_approved', 'future_default', 'total_payments_made', 'risk_category']].head())

In [ ]:
# ❌ VULNERABILIDAD 6: Sin reproducibilidad (no random seeds)
# Preparar datos para modelo
features = ['income', 'age', 'credit_score', 'future_default', 'total_payments_made']  # Incluye leakage!
X = customers[features]
y = customers['credit_approved']

# ❌ Sin random_state - resultados no reproducibles
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

In [ ]:
# ❌ VULNERABILIDAD 7: Sin random seed en modelo
# Entrenar modelo sin reproducibilidad
model = RandomForestClassifier(n_estimators=100)  # ❌ Sin random_state
model.fit(X_train, y_train)

# Evaluar
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train accuracy: {train_score:.4f}")
print(f"Test accuracy: {test_score:.4f}")
print("\n⚠️ NOTA: Estos resultados NO son reproducibles debido a falta de random seeds")

In [ ]:
# ❌ VULNERABILIDAD 8: Uso peligroso de eval()
# Feature engineering dinámico con eval
feature_formula = "income * 0.3 + age * 100"  # En producción vendría de config/usuario

# ❌ eval() permite ejecución de código arbitrario
customers['dynamic_score'] = customers.apply(
    lambda row: eval(feature_formula, {}, {'income': row['income'], 'age': row['age']}),
    axis=1
)

# Un atacante podría inyectar: "__import__('os').system('rm -rf /')"
print(f"\nDynamic score calculado con eval(): {customers['dynamic_score'].head()}")
print("⚠️ VULNERABILIDAD: eval() permite code injection!")

In [ ]:
# ❌ VULNERABILIDAD 9: Pickle inseguro
import pickle

# Guardar modelo con pickle (sin validación)
model_path = 'credit_model.pkl'

with open(model_path, 'wb') as f:
    pickle.dump(model, f)

print(f"✅ Modelo guardado en {model_path}")

# ❌ Cargar modelo sin validar integridad
def load_model_unsafe(path):
    """Carga modelo sin validación - VULNERABLE"""
    with open(path, 'rb') as f:
        return pickle.load(f)  # ❌ Sin verificar hash/signature

# Un atacante podría reemplazar el .pkl con código malicioso
loaded_model = load_model_unsafe(model_path)
print("⚠️ Modelo cargado sin validación de integridad")

In [ ]:
# ❌ VULNERABILIDAD 10: Exec con input de usuario
# Generar reporte dinámico
report_code = """
print('=== REPORTE DE MODELO ===')
print(f'Train Accuracy: {train_score}')
print(f'Test Accuracy: {test_score}')
"""

# ❌ exec() ejecuta código arbitrario
exec(report_code)  # En producción podría venir de config/usuario
print("\n⚠️ VULNERABILIDAD: exec() permite ejecución de código!")

In [ ]:
# ❌ VULNERABILIDAD 11: Logging excesivo con información sensible
import logging

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# ❌ Log con PII y secrets
logger.info(f"Procesando cliente: {customers.iloc[0]['name']}")
logger.debug(f"DNI: {customers.iloc[0]['dni']}")
logger.debug(f"API Key: {OPENAI_API_KEY}")
logger.debug(f"DB Password: {DB_PASSWORD}")

print("\n⚠️ VULNERABILIDAD: Logs contienen PII y secrets!")

In [ ]:
# ❌ VULNERABILIDAD 12: Dependencias sin versión fija
print("\n=== DEPENDENCIAS ===")
print("pandas")
print("numpy")
print("scikit-learn")
print("\n⚠️ PROBLEMA: Sin versiones fijas en requirements.txt")
print("Riesgo: Supply chain attacks, breaking changes")

---

## 🎯 RESUMEN DE VULNERABILIDADES

| # | Vulnerabilidad | Severidad | Línea/Celda |
|---|----------------|-----------|-------------|
| 1 | API Keys hardcoded | 🔴 CRÍTICA | Celda 2 |
| 2 | DB credentials hardcoded | 🔴 CRÍTICA | Celda 3 |
| 3 | SQL Injection | 🔴 CRÍTICA | Celda 4 |
| 4 | PII expuesta en logs | 🔴 CRÍTICA | Celda 5 |
| 5 | Data leakage | 🟠 ALTA | Celda 6 |
| 6 | Sin reproducibilidad | 🟠 ALTA | Celda 7 |
| 7 | Sin random seeds | 🟠 ALTA | Celda 8 |
| 8 | Uso de eval() | 🔴 CRÍTICA | Celda 9 |
| 9 | Pickle inseguro | 🟠 ALTA | Celda 10 |
| 10 | Uso de exec() | 🔴 CRÍTICA | Celda 11 |
| 11 | Logging excesivo | 🟠 ALTA | Celda 12 |
| 12 | Deps sin versión | 🟡 MEDIA | Celda 13 |

---

## 📋 EJERCICIO

**Tarea:** Usando GitHub Copilot y GHAS:

1. Ejecuta CodeQL en este notebook
2. Identifica todas las vulnerabilidades
3. Para cada una:
   - Explica el riesgo
   - Propón corrección
   - Implementa la solución
4. Crea versión segura: `secure_ml_notebook.ipynb`

**Tiempo estimado:** 30 minutos

**Prompts recomendados:** Ver `LIBRERIA_PROMPTS_GHAS.md`